In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
#from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import * 

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 23), datetime.date(2023, 6, 22))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-06-23
Today in mm-dd format: 06-23
yesterday: 2023-06-22 00:00:00


In [3]:
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2023-05-25 00:00:00   Thursday


In [4]:
format_dict = {
    'ttl_qty':'{:,}','qty':'{:,}',
    'from_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}','fm_price':'{:.2f}',
    'price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','pct':'{:,.2f}%',  
}

cols = 'name fm_md to_md days fm_price to_price pct spd avg max_price min_price ttl_qty trend'.split()

### Get past one quarter data

In [5]:
sql = """
SELECT name
FROM buy 
ORDER BY name
"""
buy = pd.read_sql(sql, const)

names = buy['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TMT', 'WHAIR', 'WHART'"

In [6]:
in_p = in_p + ", 'TFFIF', 'BEM', 'EA'"
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TMT', 'WHAIR', 'WHART', 'TFFIF', 'BEM', 'EA'"

### Bypass this statement if aboce statement is executed

In [8]:
sql = """
SELECT name
FROM orders
ORDER BY trade, name
"""
#buy = pd.read_sql(sql, conlite)

#names = buy['name'].values.tolist()
#in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'AWC', 'GULF', 'TTB'"

### End of Selection between Buy or Orders

In [7]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_month_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-05-25 00:00:00' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TMT', 'WHAIR', 'WHART', 'TFFIF', 'BEM', 'EA') 
ORDER BY name, date


In [8]:
price = pd.read_sql(sql, const)
price.sample(5)

,name,date,price,qty,maxp,minp
537,SINGER,2023-06-13,11.10,6992288,11.30,10.90
122,BCH,2023-06-20,18.20,4024626,18.30,18.00
120,BCH,2023-06-16,18.00,25490705,18.50,17.70
327,JMT,2023-06-13,43.25,8666627,43.75,43.00
142,BEM,2023-06-19,8.30,10491203,8.35,8.25


In [9]:
data_path = "../data/"
file_name = "Quarterly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

price.to_csv(output_file, header=None, index=False)
price.to_csv(osd_file, header=True, index=False)

### Call ruby ruby\crt-summary.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\crt-summary.rb

ruby/../Data/Quarterly-Price-by-Name.csv
name|fm_date|to_date|days|fm_price|to_price|diff|pct|spd|avg|max_price|min_price|trend|nbr|ttl_qty
AH|2023-05-25|2023-05-25|1|34.0|34.0|0.0|0.0|0|0|34.5|33.75||2|554,612
AH|2023-05-26|2023-05-31|4|34.75|37.75|3.0|8.63|12|3|38.25|34.0|Upward|6|14,202,687
AH|2023-06-01|2023-06-01|1|37.0|37.0|0.0|0.0|0|0|37.75|36.5|Downward|7|2,882,866
AH|2023-06-02|2023-06-07|4|37.25|39.25|2.0|5.37|8|2|39.25|36.25|Upward|10|8,078,969
AH|2023-06-08|2023-06-09|2|36.5|34.5|-2.0|-5.48|-8|-4|37.75|27.75|Downward|12|103,830,264
AH|2023-06-12|2023-06-12|1|35.25|35.25|0.0|0.0|0|0|36.25|34.75|Upward|13|7,101,473
AH|2023-06-13|2023-06-14|2|35.0|34.5|-0.5|-1.43|-2|-1|35.75|33.75|Downward|15|5,935,722
AH|2023-06-15|2023-06-15|1|34.75|34.75|0.0|0.0|0|0|35.25|34.5|Upward|16|1,501,200
AH|2023-06-16|2023-06-21|4|34.5|32.75|-1.75|-5.07|-7|-2|35.0|32.25|Downward|20|8,367,251
AH|2023-06-22|2023-06-22|1|33.25|33.25|0.0|0.0|0|0|33.75|32.25|Upward|21|2,011,971
AH|2023-06-23|2023-06-23|

JASIF|2023-06-12|2023-06-12|1|6.95|6.95|0.0|0.0|0|0|7.0|6.95|Downward|286|2,209,493
JASIF|2023-06-13|2023-06-16|4|7.0|7.4|0.4|5.71|8|2|7.45|6.95|Upward|290|46,057,345
JASIF|2023-06-19|2023-06-23|5|7.25|7.1|-0.15|-2.07|-3|-1|7.45|7.0|Downward|295|42,789,330
JMART|2023-05-25|2023-05-26|2|20.7|19.1|-1.6|-7.73|-16|-8|21.1|19.0|Downward|297|75,556,859
JMART|2023-05-29|2023-05-31|3|19.3|20.4|1.1|5.7|11|3|20.5|19.0|Upward|300|66,871,020
JMART|2023-06-01|2023-06-07|5|20.1|19.2|-0.9|-4.48|-9|-2|20.5|18.9|Downward|304|56,896,458
JMART|2023-06-08|2023-06-14|5|19.3|20.1|0.8|4.15|8|1|20.5|18.9|Upward|309|82,009,899
JMART|2023-06-15|2023-06-23|7|19.3|16.0|-3.3|-17.1|-33|-5|20.1|15.6|Downward|316|170,215,757
JMT|2023-05-25|2023-05-26|2|42.0|40.5|-1.5|-3.57|-6|-3|42.25|40.5|Downward|318|15,133,304
JMT|2023-05-29|2023-05-29|1|40.75|40.75|0.0|0.0|0|0|41.75|40.25|Upward|319|5,354,030
JMT|2023-05-30|2023-05-30|1|39.75|39.75|0.0|0.0|0|0|40.75|39.5|Downward|320|11,998,586
JMT|2023-05-31|2023-05-31|1|43.0|43

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'summary.csv'
input_file = data_path + file_name
df = pd.read_csv(input_file, sep=',', index_col=None)
df.sample(5).style.format(format_dict)

,name,from_date,to_date,days,from_price,to_price,diff,pct,spd,avg,max_price,min_price,ttl_qty,trend,nbr
276,WHAIR,2023-06-02,2023-06-02,1,7.10,7.10,0.00,0.00%,0,0,7.20,7.10,"485,216",Downward,638
141,JMT,2023-06-21,2023-06-23,3,38.75,36.75,-2.00,-5.16%,-8,-3,41.00,36.25,"46,779,645",Downward,337
259,TFFIF,2023-06-12,2023-06-14,3,7.60,7.65,0.05,0.66%,1,0,7.65,7.50,"15,323,520",Upward,603
46,BANPU,2023-06-19,2023-06-19,1,9.00,9.00,0.00,0.00%,0,0,9.05,8.95,"60,685,294",Downward,102
262,TFFIF,2023-06-19,2023-06-19,1,7.70,7.70,0.00,0.00%,0,0,7.80,7.70,"1,292,943",Downward,606


In [15]:
df.rename(columns={'from_date':'fm_date','from_price':'fm_price'},inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['fm_md'] = df['fm_date'].dt.strftime('%m-%d')
df['to_md'] = df['to_date'].dt.strftime('%m-%d')

### Group analysis

In [16]:
mask_p = df['to_md'] == to_mmdd
df[mask_p][cols].sort_values('pct',ascending=False).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
78,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward
49,BANPU,06-22,06-23,2,9.10,9.15,0.55%,1,0,9.15,9.00,"109,755,280",Upward
162,KTC,06-20,06-23,4,50.25,50.50,0.50%,1,0,51.25,49.50,"15,842,414",Upward
10,AH,06-23,06-23,1,32.25,32.25,0.00%,0,0,33.00,32.25,"2,130,807",Downward
265,TFFIF,06-23,06-23,1,7.70,7.70,0.00%,0,0,7.70,7.55,"2,910,530",Upward


In [17]:
name_up = 'CPNREIT'
mask_n = df.name == name_up
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
71,CPNREIT,05-25,05-26,2,12.90,12.80,-0.78%,-1,-1,12.90,12.70,"1,167,086",Downward
72,CPNREIT,05-29,05-30,2,12.90,12.90,0.00%,0,0,12.90,12.80,"811,213",Upward
73,CPNREIT,05-31,06-02,3,12.80,12.80,0.00%,0,0,12.90,12.70,"2,250,814",Downward
74,CPNREIT,06-06,06-07,2,12.90,12.90,0.00%,0,0,13.00,12.60,"3,678,090",Upward
75,CPNREIT,06-08,06-08,1,12.80,12.80,0.00%,0,0,12.90,12.80,"795,603",Downward
76,CPNREIT,06-09,06-09,1,13.00,13.00,0.00%,0,0,13.00,12.70,"2,498,945",Upward
77,CPNREIT,06-12,06-21,8,12.90,10.10,-21.71%,-28,-4,13.00,10.10,"46,450,088",Downward
78,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward


In [18]:
mask_m = df['to_md'] == to_mmdd
df[mask_m][cols].sort_values('pct',ascending=True).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
132,JMART,06-15,06-23,7,19.30,16.00,-17.10%,-33,-5,20.10,15.60,"170,215,757",Downward
82,DIF,06-09,06-23,11,12.30,10.60,-13.82%,-17,-2,12.40,10.00,"416,432,342",Downward
244,STA,06-19,06-23,5,18.80,17.10,-9.04%,-17,-4,19.30,17.00,"42,123,028",Downward
111,ICHI,06-15,06-23,7,15.00,14.10,-6.00%,-9,-2,15.40,13.80,"62,416,892",Downward
141,JMT,06-21,06-23,3,38.75,36.75,-5.16%,-8,-3,41.00,36.25,"46,779,645",Downward


In [19]:
name_down = 'JMART'
mask_n = df.name == name_down
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
128,JMART,05-25,05-26,2,20.70,19.10,-7.73%,-16,-8,21.10,19.00,"75,556,859",Downward
129,JMART,05-29,05-31,3,19.30,20.40,5.70%,11,3,20.50,19.00,"66,871,020",Upward
130,JMART,06-01,06-07,5,20.10,19.20,-4.48%,-9,-2,20.50,18.90,"56,896,458",Downward
131,JMART,06-08,06-14,5,19.30,20.10,4.15%,8,1,20.50,18.90,"82,009,899",Upward
132,JMART,06-15,06-23,7,19.30,16.00,-17.10%,-33,-5,20.10,15.60,"170,215,757",Downward


In [20]:
df_groupby_trend = df.groupby('trend')

In [21]:
# To retrieve one of the created groups
df_up = df_groupby_trend.get_group('Upward')
df_up[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
23,ASP,06-07,06-09,3,2.86,2.88,0.70%,1,0,2.88,2.84,"1,555,630",Upward
257,TFFIF,05-26,05-29,2,7.75,7.75,0.00%,0,0,7.75,7.70,"2,487,860",Upward
232,SINGER,06-14,06-14,1,11.30,11.30,0.00%,0,0,11.60,11.10,"8,700,370",Upward
104,ICHI,05-31,06-01,2,14.50,14.70,1.38%,2,1,14.80,14.30,"15,770,816",Upward
160,KTC,06-16,06-16,1,50.25,50.25,0.00%,0,0,50.50,49.25,"5,252,490",Upward


In [22]:
# To retrieve one of the created groups
df_dw = df_groupby_trend.get_group('Downward')
df_dw[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
217,SCC,06-21,06-21,1,325.00,325.00,0.00%,0,0,330.00,325.00,"1,643,301",Downward
278,WHAIR,06-12,06-13,2,7.25,7.25,0.00%,0,0,7.30,7.20,"154,804",Downward
128,JMART,05-25,05-26,2,20.70,19.10,-7.73%,-16,-8,21.10,19.00,"75,556,859",Downward
109,ICHI,06-13,06-13,1,15.00,15.00,0.00%,0,0,15.40,14.90,"11,995,449",Downward
55,BCH,06-09,06-09,1,18.30,18.30,0.00%,0,0,18.60,18.30,"14,152,226",Downward


In [23]:
# there is a function called .agg() and it allows us to specify multiple aggregation functions at once
df.groupby('trend').days.agg(['max', 'count', 'median', 'mean'])

,max,count,median,mean
trend,,,,
Downward,11,153,2.00,2.66
Upward,9,137,2.00,2.04


In [24]:
upward_max = df.groupby('trend').days.max().loc['Upward']
downward_max = df.groupby('trend').days.max().loc['Downward']

In [25]:
df.query('days == @upward_max and trend == "Upward"')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
205,RCL,06-02,06-14,9,23.70,25.00,5.49%,13,1,25.25,23.30,"31,347,151",Upward


In [26]:
df.query('days == @downward_max and trend == "Downward"')[cols]

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
82,DIF,06-09,06-23,11,12.30,10.60,-13.82,-17,-2,12.40,10.00,416432342,Downward


### Individual stock analysis

In [27]:
df.query('name == @name_up')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
71,CPNREIT,05-25,05-26,2,12.90,12.80,-0.78%,-1,-1,12.90,12.70,"1,167,086",Downward
72,CPNREIT,05-29,05-30,2,12.90,12.90,0.00%,0,0,12.90,12.80,"811,213",Upward
73,CPNREIT,05-31,06-02,3,12.80,12.80,0.00%,0,0,12.90,12.70,"2,250,814",Downward
74,CPNREIT,06-06,06-07,2,12.90,12.90,0.00%,0,0,13.00,12.60,"3,678,090",Upward
75,CPNREIT,06-08,06-08,1,12.80,12.80,0.00%,0,0,12.90,12.80,"795,603",Downward
76,CPNREIT,06-09,06-09,1,13.00,13.00,0.00%,0,0,13.00,12.70,"2,498,945",Upward
77,CPNREIT,06-12,06-21,8,12.90,10.10,-21.71%,-28,-4,13.00,10.10,"46,450,088",Downward
78,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward


In [28]:
df_tmp = price.query('name == @name_up')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,21.00,21.00,21.00,21.00
mean,12.24,3235339.86,12.42,12.05
std,0.98,3725696.80,0.85,1.07
min,10.10,97082.00,10.50,9.90
25%,12.00,685031.00,12.30,11.80
50%,12.80,1533485.00,12.90,12.70
75%,12.90,4383653.00,12.90,12.80
max,13.00,13675377.00,13.00,12.80


In [29]:
df.query('name == @name_down')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
128,JMART,05-25,05-26,2,20.70,19.10,-7.73%,-16,-8,21.10,19.00,"75,556,859",Downward
129,JMART,05-29,05-31,3,19.30,20.40,5.70%,11,3,20.50,19.00,"66,871,020",Upward
130,JMART,06-01,06-07,5,20.10,19.20,-4.48%,-9,-2,20.50,18.90,"56,896,458",Downward
131,JMART,06-08,06-14,5,19.30,20.10,4.15%,8,1,20.50,18.90,"82,009,899",Upward
132,JMART,06-15,06-23,7,19.30,16.00,-17.10%,-33,-5,20.10,15.60,"170,215,757",Downward


In [30]:
df_tmp = price.query('name == @name_down')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,21.00,21.00,21.00,21.00
mean,19.12,21502380.62,19.67,18.84
std,1.31,14088101.40,1.15,1.26
min,16.00,8185881.00,16.50,15.60
25%,19.10,13152640.00,19.40,18.90
50%,19.30,15909129.00,20.10,19.10
75%,19.90,23528362.00,20.30,19.60
max,20.70,59073442.00,21.10,19.90


In [31]:
mask_p = df['to_md'] == to_mmdd
summary = df[mask_p][cols].sort_values(['pct','trend'],ascending=[False,False])

In [32]:
file_name = 'summary-240.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

summary[cols].to_csv(output_file, header=True, index=False)
summary[cols].to_csv(data_file, header=True, index=False)
summary[cols].to_csv(one_file, header=True, index=False)
summary[cols].to_csv(osd_file, header=True, index=False)

In [33]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-06-23' 
ORDER BY name, date


In [34]:
df = pd.read_sql(sql, const)
df

,name,date,price,qty,maxp,minp
0,ACE,2023-06-23,1.89,15610982,1.92,1.88
1,ADVANC,2023-06-23,211.00,4423809,213.00,209.00
2,AEONTS,2023-06-23,186.00,213604,190.50,185.50
3,AH,2023-06-23,32.25,2130807,33.00,32.25
4,AIE,2023-06-23,1.44,212273,1.44,1.42
...,...,...,...,...,...,...
187,WHAIR,2023-06-23,7.20,271400,7.25,7.15
188,WHART,2023-06-23,10.60,2918600,10.70,10.40
189,WHAUP,2023-06-23,3.74,837232,3.78,3.72
190,WICE,2023-06-23,8.45,937825,8.55,8.35
